### Triplet Loss Scaffold

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
from collections import defaultdict

class TripletDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        # Create a dictionary mapping from label to indices with that label
        self.label_to_indices = defaultdict(list)
        for idx, label in enumerate(labels):
            self.label_to_indices[label].append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anchor = self.data[idx]
        anchor_label = self.labels[idx]
        positive_idx = idx
        while positive_idx == idx:
            positive_idx = np.random.choice(self.label_to_indices[anchor_label])
        positive = self.data[positive_idx]
        
        # Ensure the negative label is different from the anchor label
        negative_label = anchor_label
        while negative_label == anchor_label:
            negative_label = np.random.choice(list(self.label_to_indices.keys()))
        negative_idx = np.random.choice(self.label_to_indices[negative_label])
        negative = self.data[negative_idx]

        return anchor, positive, negative

# Example data
data = np.random.randn(100, 3, 32, 32)  # 100 samples of 3x32x32 images
labels = np.random.randint(0, 10, 100)  # 100 random labels ranging from 0 to 9

# Create the dataset
triplet_dataset = TripletDataset(data, labels)
triplet_loader = torch.utils.data.DataLoader(triplet_dataset, batch_size=32, shuffle=True)

# Iterate through the dataset
for anchors, positives, negatives in triplet_loader:
    # Your training code here
    pass

In [ ]:
import torch
import torch.nn as nn

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        positive_distance = torch.nn.functional.pairwise_distance(anchor, positive)
        negative_distance = torch.nn.functional.pairwise_distance(anchor, negative)
        loss = torch.mean(torch.relu(positive_distance - negative_distance + self.margin))
        return loss

# Example usage:
# loss_fn = TripletLoss(margin=1.0)
# anchor, positive, negative are the outputs from the network for the anchor, positive, and negative samples respectively

### Minimal Triplet Loss Application

In [ ]:
import torch
from torch.utils.data import Dataset
import random
from sklearn.feature_extraction.text import CountVectorizer

class TextTripletDataset(Dataset):
    def __init__(self, phrases, labels, transform=None):
        """
        Args:
            phrases (list): List of text phrases.
            labels (list): List of corresponding labels.
            transform (callable, optional): Optional transform to be applied on the text sample.
        """
        self.phrases = phrases
        self.labels = labels
        self.transform = transform
        self.label_to_indices = self._create_label_to_indices()
        self.vectorizer = CountVectorizer()
        self.vectorized_phrases = self.vectorizer.fit_transform(phrases).toarray()

    def _create_label_to_indices(self):
        """Creates a dictionary to map labels to indices of samples with that label."""
        label_to_indices = {}
        for index, label in enumerate(self.labels):
            if label not in label_to_indices:
                label_to_indices[label] = []
            label_to_indices[label].append(index)
        return label_to_indices

    def __len__(self):
        return len(self.phrases)

    def __getitem__(self, index):
        anchor_phrase_vector = self.vectorized_phrases[index]
        anchor_label = self.labels[index]
        
        # Set positive sample (same label)
        positive_index = index
        while positive_index == index:
            positive_index = random.choice(self.label_to_indices[anchor_label])
        positive_phrase_vector = self.vectorized_phrases[positive_index]

        # Set negative sample (different label)
        negative_label = random.choice(list(self.label_to_indices.keys()))
        while negative_label == anchor_label:
            negative_label = random.choice(list(self.label_to_indices.keys()))
        negative_index = random.choice(self.label_to_indices[negative_label])
        negative_phrase_vector = self.vectorized_phrases[negative_index]

        return (torch.tensor(anchor_phrase_vector, dtype=torch.float32),
                torch.tensor(positive_phrase_vector, dtype=torch.float32),
                torch.tensor(negative_phrase_vector, dtype=torch.float32))

# Example usage:
# phrases = ["I love dogs", "I love cats", "I hate dogs", "I hate cats"]
# labels = [0, 1, 2, 3]  # Assign numerical labels
# dataset = TextTripletDataset(phrases, labels)
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        positive_distance = F.pairwise_distance(anchor, positive)
        negative_distance = F.pairwise_distance(anchor, negative)
        losses = torch.relu(positive_distance - negative_distance + self.margin)
        return losses.mean()

# Example usage with a sample model:
# model = SampleModel()  # Define your model here
# criterion = TripletLoss(margin=1.0)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# for epoch in range(num_epochs):
#     for anchor, positive, negative in dataloader:
#         anchor_output = model(anchor)
#         positive_output = model(positive)
#         negative_output = model(negative)
#         loss = criterion(anchor_output, positive_output, negative_output)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

### Contrastive Loss Scaffold

In [ ]:
import torch
from torch.utils.data import Dataset
import random

class ContrastiveDataset(Dataset):
    def __init__(self, data, transform=None):
        """
        Args:
            data (list of tuples): List of (image, label) tuples.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = data
        self.transform = transform
        self.label_to_indices = self._create_label_to_indices()

    def _create_label_to_indices(self):
        """Creates a dictionary to map labels to indices of samples with that label."""
        label_to_indices = {}
        for index, (_, label) in enumerate(self.data):
            if label not in label_to_indices:
                label_to_indices[label] = []
            label_to_indices[label].append(index)
        return label_to_indices

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img1, label1 = self.data[index]
        
        # Randomly decide to pick a positive or negative pair
        if random.random() < 0.5:
            # Positive sample
            positive_index = index
            while positive_index == index:
                positive_index = random.choice(self.label_to_indices[label1])
            img2, label2 = self.data[positive_index]
        else:
            # Negative sample
            negative_label = random.choice(list(self.label_to_indices.keys()))
            while negative_label == label1:
                negative_label = random.choice(list(self.label_to_indices.keys()))
            negative_index = random.choice(self.label_to_indices[negative_label])
            img2, label2 = self.data[negative_index]

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), (label1, label2)


In [ ]:
import torch
import torch.nn as nn

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean(
            (1 - label) * torch.pow(euclidean_distance, 2) +
            (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        )
        return loss_contrastive

# Example usage:
# loss_fn = ContrastiveLoss(margin=1.0)
# output1, output2 are the outputs from the two branches of the Siamese network
# label is 0 if the inputs are from the same class and 1 otherwise

### Minimal Contrastive Loss Application

In [ ]:
import torch
from torch.utils.data import Dataset
import random
from sklearn.feature_extraction.text import CountVectorizer

class TextContrastiveDataset(Dataset):
    def __init__(self, phrases, labels, transform=None):
        """
        Args:
            phrases (list): List of text phrases.
            labels (list): List of corresponding labels.
            transform (callable, optional): Optional transform to be applied on the text sample.
        """
        self.phrases = phrases
        self.labels = labels
        self.transform = transform
        self.label_to_indices = self._create_label_to_indices()
        self.vectorizer = CountVectorizer()
        self.vectorized_phrases = self.vectorizer.fit_transform(phrases).toarray()

    def _create_label_to_indices(self):
        """Creates a dictionary to map labels to indices of samples with that label."""
        label_to_indices = {}
        for index, label in enumerate(self.labels):
            if label not in label_to_indices:
                label_to_indices[label] = []
            label_to_indices[label].append(index)
        return label_to_indices

    def __len__(self):
        return len(self.phrases)

    def __getitem__(self, index):
        phrase1 = self.vectorized_phrases[index]
        label1 = self.labels[index]
        
        # Randomly decide to pick a positive or negative pair
        if random.random() < 0.5:
            # Positive sample
            positive_index = index
            while positive_index == index:
                positive_index = random.choice(self.label_to_indices[label1])
            phrase2 = self.vectorized_phrases[positive_index]
            label2 = label1
        else:
            # Negative sample
            negative_label = random.choice(list(self.label_to_indices.keys()))
            while negative_label == label1:
                negative_label = random.choice(list(self.label_to_indices.keys()))
            negative_index = random.choice(self.label_to_indices[negative_label])
            phrase2 = self.vectorized_phrases[negative_index]
            label2 = negative_label

        return (torch.tensor(phrase1, dtype=torch.float32), torch.tensor(phrase2, dtype=torch.float32)), torch.tensor(int(label1 == label2), dtype=torch.float32)

# Example usage:
# phrases = ["I love dogs", "I love cats", "I hate dogs", "I hate cats"]
# labels = [0, 1, 2, 3]  # Assign numerical labels
# dataset = TextContrastiveDataset(phrases, labels)
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

# Example usage with a sample model:
# model = SampleModel()  # Define your model here
# criterion = ContrastiveLoss(margin=1.0)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# for epoch in range(num_epochs):
#     for (data1, data2), label in dataloader:
#         output1 = model(data1)
#         output2 = model(data2)
#         loss = criterion(output1, output2, label)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()